# **Projet étape 1**

Importation des bibliothèques

In [4]:
import requests
import pandas as pd
import numpy as np

1. Analyse descriptive des données

Les données utilisées dans ce projet proviennent de la base publique de la NVD (National Vulnerability Database). Chaque entrée correspond à une vulnérabilité référencée par un identifiant CVE unique. Les informations disponibles sont hétérogènes : description textuelle de la vulnérabilité, métadonnées contextuelles, liens externes, tags, dates de publication, ainsi que différents scores d’impact (CVSS). La distribution de la variable cible (la sévérité) est déséquilibrée, avec une majorité de vulnérabilités classées MEDIUM et HIGH, tandis que les classes LOW et CRITICAL sont nettement moins représentées. Ce déséquilibre statistique est fréquent dans les jeux de données de cybersécurité et aura un impact important sur la performance des modèles supervisés, nécessitant des techniques d’ajustement spécifiques. Cette première étape descriptive permet d’identifier la nature des variables, leur format, ainsi que les contraintes structurelles du dataset avant tout traitement automatique.

Importation du dataset après filtrage

In [5]:
import json
 
file_path = "data_iot.json"

with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)
 
vulns = data.get("vulnerabilities", [])
print(f" Loaded {len(vulns):,} IoT vulnerability entries from file.")
 
df = pd.json_normalize(vulns)
 
print(f" DataFrame created: {df.shape[0]:,} rows and {df.shape[1]} columns")
 
df.head(3)

 Loaded 8,833 IoT vulnerability entries from file.
 DataFrame created: 8,833 rows and 22 columns


,cve.id,cve.sourceIdentifier,cve.published,cve.lastModified,cve.vulnStatus,cve.cveTags,cve.descriptions,cve.metrics.cvssMetricV2,cve.weaknesses,cve.configurations,...,cve.vendorComments,cve.evaluatorSolution,cve.evaluatorComment,cve.evaluatorImpact,cve.cisaExploitAdd,cve.cisaActionDue,cve.cisaRequiredAction,cve.cisaVulnerabilityName,cve.metrics.cvssMetricV30,cve.metrics.cvssMetricV40
0,CVE-1999-0257,cve@mitre.org,1998-04-01T05:00:00.000,2025-04-03T01:03:51.193,Deferred,[],"[{'lang': 'en', 'value': 'Nestea variation of ...","[{'source': 'nvd@nist.gov', 'type': 'Primary',...","[{'source': 'nvd@nist.gov', 'type': 'Primary',...","[{'nodes': [{'operator': 'OR', 'negate': False...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CVE-1999-1499,cve@mitre.org,1998-04-10T04:00:00.000,2025-04-03T01:03:51.193,Deferred,[],"[{'lang': 'en', 'value': 'named in ISC BIND 4....","[{'source': 'nvd@nist.gov', 'type': 'Primary',...","[{'source': 'nvd@nist.gov', 'type': 'Primary',...","[{'nodes': [{'operator': 'OR', 'negate': False...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CVE-1999-1292,cve@mitre.org,1998-09-01T04:00:00.000,2025-04-03T01:03:51.193,Deferred,[],"[{'lang': 'en', 'value': 'Buffer overflow in w...","[{'source': 'nvd@nist.gov', 'type': 'Primary',...","[{'source': 'nvd@nist.gov', 'type': 'Primary',...","[{'nodes': [{'operator': 'OR', 'negate': False...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
import pandas as pd

# 1) sanity check: see what we actually have
print(sorted(df.columns.tolist())[:50])   # confirm 'metrics' vs 'impact' vs 'cve.metrics'

# 2) description (unchanged)
def get_en(desc):
    if isinstance(desc, (list, tuple)):
        for d in desc:
            if isinstance(d, dict) and d.get("lang") == "en":
                return d.get("value")
    return np.nan

df["description_en"] = df["cve.descriptions"].apply(get_en)

# 3) robust CVSS extractor with fallbacks
def extract_cvss_fields(row):
    # pick the column that exists
    metrics_obj = None
    for col in ("metrics", "cve.metrics", "impact"):
        if col in row and pd.notna(row[col]):
            metrics_obj = row[col]
            break

    sev = np.nan
    score3 = np.nan
    score2 = np.nan

    # NVD 2.0 style: metrics is a dict with lists cvssMetricV31 / V30 / V2
    if isinstance(metrics_obj, dict):
        for k in ("cvssMetricV31", "cvssMetricV30"):
            lst = metrics_obj.get(k)
            if isinstance(lst, list) and lst:
                data = (lst[0] or {}).get("cvssData", {})
                sev = data.get("baseSeverity", sev)
                score3 = data.get("baseScore", score3)
                break
        # try v2 if v3 not present
        if pd.isna(score3):
            lst2 = metrics_obj.get("cvssMetricV2")
            if isinstance(lst2, list) and lst2:
                data2 = (lst2[0] or {}).get("cvssData", {})
                score2 = data2.get("baseScore", score2)

    # Older schema: 'impact' with 'baseMetricV2'
    if pd.isna(score3) and isinstance(metrics_obj, dict):
        bm2 = metrics_obj.get("baseMetricV2")  # older JSON
        if isinstance(bm2, dict):
            cv2 = bm2.get("cvssV2", {})
            score2 = cv2.get("baseScore", score2)

    # derive severity from v2 if v3 missing
    if pd.isna(sev):
        if pd.notna(score2):
            s = float(score2)
            if s < 4.0: sev = "LOW"
            elif s < 7.0: sev = "MEDIUM"
            elif s < 9.0: sev = "HIGH"
            else: sev = "CRITICAL"

    return pd.Series({"cvss3_score": score3, "cvss2_score": score2, "severity": sev})

df[["cvss3_score", "cvss2_score", "severity"]] = df.apply(extract_cvss_fields, axis=1)

# Keep what you need
out = df[["cve.id", "description_en", "severity", "cvss3_score", "cvss2_score", "cve.published", "cve.lastModified"]]
print(out.head())

['cve.cisaActionDue', 'cve.cisaExploitAdd', 'cve.cisaRequiredAction', 'cve.cisaVulnerabilityName', 'cve.configurations', 'cve.cveTags', 'cve.descriptions', 'cve.evaluatorComment', 'cve.evaluatorImpact', 'cve.evaluatorSolution', 'cve.id', 'cve.lastModified', 'cve.metrics.cvssMetricV2', 'cve.metrics.cvssMetricV30', 'cve.metrics.cvssMetricV31', 'cve.metrics.cvssMetricV40', 'cve.published', 'cve.references', 'cve.sourceIdentifier', 'cve.vendorComments', 'cve.vulnStatus', 'cve.weaknesses']
          cve.id                                     description_en  severity  \
0  CVE-1999-0257  Nestea variation of teardrop IP fragmentation ...       NaN   
1  CVE-1999-1499  named in ISC BIND 4.9 and 8.1 allows local use...       NaN   
2  CVE-1999-1292  Buffer overflow in web administration feature ...       NaN   
3  CVE-1999-0911  Buffer overflow in ProFTPD, wu-ftpd, and berof...       NaN   
4  CVE-1999-1513  Management information base (MIB) for a 3Com S...       NaN   

   cvss3_score  cvss2_s

In [7]:

def extract_cvss(row):

    # CVSS v3.1
    m31 = row.get("cve.metrics.cvssMetricV31")
    if isinstance(m31, list) and len(m31)>0:
        d = m31[0].get("cvssData", {})
        sev = d.get("baseSeverity")
        score3 = d.get("baseScore")
        return sev, score3, np.nan
    
    # CVSS v3.0
    m30 = row.get("cve.metrics.cvssMetricV30")
    if isinstance(m30, list) and len(m30)>0:
        d = m30[0].get("cvssData", {})
        sev = d.get("baseSeverity")
        score3 = d.get("baseScore")
        return sev, score3, np.nan
    
    # CVSS v2 fallback
    m2 = row.get("cve.metrics.cvssMetricV2")
    if isinstance(m2, list) and len(m2)>0:
        d = m2[0].get("cvssData", {})
        score2 = d.get("baseScore")
        if score2 is not None:
            # convertir score en niveau
            if score2 < 4: sev="LOW"
            elif score2 < 7: sev="MEDIUM"
            elif score2 < 9: sev="HIGH"
            else: sev="CRITICAL"
            return sev, np.nan, score2
    
    return np.nan, np.nan, np.nan

df[["severity","cvss3_score","cvss2_score"]] = df.apply(lambda r: pd.Series(extract_cvss(r)), axis=1)


In [8]:
df_final = df[ ["cve.id","description_en","severity","cvss3_score","cvss2_score","cve.published","cve.lastModified"] ]
df_final = df_final.dropna(subset=["description_en","severity"]).reset_index(drop=True)

df_final.head()

,cve.id,description_en,severity,cvss3_score,cvss2_score,cve.published,cve.lastModified
0,CVE-1999-0257,Nestea variation of teardrop IP fragmentation ...,MEDIUM,NaN,5.0,1998-04-01T05:00:00.000,2025-04-03T01:03:51.193
1,CVE-1999-1499,named in ISC BIND 4.9 and 8.1 allows local use...,LOW,NaN,2.1,1998-04-10T04:00:00.000,2025-04-03T01:03:51.193
2,CVE-1999-1292,Buffer overflow in web administration feature ...,HIGH,NaN,7.5,1998-09-01T04:00:00.000,2025-04-03T01:03:51.193
3,CVE-1999-0911,"Buffer overflow in ProFTPD, wu-ftpd, and berof...",CRITICAL,NaN,10.0,1999-08-27T04:00:00.000,2025-04-03T01:03:51.193
4,CVE-1999-1513,Management information base (MIB) for a 3Com S...,HIGH,NaN,7.5,1999-08-30T04:00:00.000,2025-04-03T01:03:51.193


Nous avons récupéré les données CVE au format JSON de la NVD. Les descriptions des vulnérabilités étant disponibles dans plusieurs langues, nous avons extrait systématiquement la version anglaise (description_en). La cible à prédire (variable de sortie) a été construite à partir des métriques CVSS disponibles (cvssMetricV31, cvssMetricV30 ou cvssMetricV2 en fallback). Les niveaux de sévérité ont été normalisés selon les valeurs officielles NVD : {LOW, MEDIUM, HIGH, CRITICAL}. Après nettoyage des champs non pertinents, on obtient un dataset final contenant : l’identifiant CVE, la description en anglais, la sévérité ainsi que les dates de création et mise à jour.

3. Formalisation du problème

Le problème étudié dans ce travail est formulé comme une tâche de classification supervisée appliquée au domaine du NLP (Natural Language Processing). L’objectif consiste à prédire automatiquement le niveau de sévérité d’une vulnérabilité à partir de sa description textuelle en anglais. La sévérité retenue correspond au niveau issu des scores CVSS fournis par la NVD, normalisé en quatre classes : LOW, MEDIUM, HIGH et CRITICAL. L’entrée du modèle est donc un texte brut descriptif, et la sortie attendue est une classe discrète reflétant l’importance de la vulnérabilité. Ce type de formalisation permet d’évaluer la capacité d’un modèle à comprendre le contenu sémantique d’une vulnérabilité et à reproduire automatiquement un jugement expert d’impact.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix


In [13]:

X = df_final["description_en"]
y = df_final["severity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.95)),
    ("clf", DecisionTreeClassifier(max_depth=None, random_state=42))
])

model.fit(X_train, y_train)
pred = model.predict(X_test)

print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

    CRITICAL       0.52      0.51      0.52       308
        HIGH       0.59      0.60      0.60       694
         LOW       0.42      0.36      0.39        64
      MEDIUM       0.66      0.66      0.66       666

    accuracy                           0.60      1732
   macro avg       0.55      0.53      0.54      1732
weighted avg       0.60      0.60      0.60      1732

[[158 103   1  46]
 [ 99 418  13 164]
 [  1  20  23  20]
 [ 44 166  18 438]]


4. Sélection d’un modèle baseline et implémentation du modèle

Pour définir une première référence expérimentale, nous avons sélectionné comme modèle baseline un arbre de décision. Ce type de modèle est particulièrement adapté pour tester rapidement des hypothèses, car il est simple à interpréter, ne nécessite pas de fortes contraintes d’hypothèses statistiques, et permet d’identifier de manière explicite les règles de décision apprises à partir du texte transformé numériquement.

Les descriptions textuelles des vulnérabilités ont été vectorisées à l’aide d’une représentation TF-IDF, permettant de convertir le texte en une matrice exploitable par le modèle. Après séparation du dataset en jeux d’entraînement et de test, le modèle DecisionTreeClassifier a été entraîné sur les données vectorisées, puis évalué sur la tâche de prédiction du niveau de sévérité.

Ce modèle baseline permet surtout de fournir un point de départ minimal permettant de quantifier la difficulté du problème et la qualité de la représentation textuelle, avant d’introduire des modèles plus avancés. Bien que l’arbre de décision soit limité en généralisation sur des données textuelles complexes, il constitue un repère initial important pour situer les performances futures. Les résultats obtenus serviront de référence comparative lors de l’évaluation d’approches plus performantes (modèles linéaires optimisés, modèles équilibrés, puis modèles Transformers).

# **Projet étape 2**

In [19]:
param_grid = {
    
    "tfidf__ngram_range": [(1,1), (1,2)],
    "tfidf__min_df": [1,3,5],
    "tfidf__max_df": [0.9, 0.95],

    "clf__criterion": ["gini", "entropy", "log_loss"],
    "clf__max_depth": [None, 10, 20, 50],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4]
}

In [ ]:
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Meilleurs paramètres :", grid.best_params_)
print("Meilleur accuracy (CV) :", grid.best_score_)

best_model = grid.best_estimator_

Meilleurs paramètres : {'clf__criterion': 'gini', 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 5, 'tfidf__max_df': 0.9, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 2)}
Meilleur accuracy (CV) : 0.598381129500263


: 